<a href="https://colab.research.google.com/github/markcam1/machine-learning-00/blob/main/Final_Project_Tech27_load-and-feature-engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clustering and Predictive Modeling of S&P 500 Stock Price Behaviors

Final Project Tech 27

In [ ]:
# S&P 500 Dataset - Complete Data Review

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set display options for better output
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

print("S&P 500 DATASET REVIEW")
print("=" * 60)
print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


## Download and Review Data

In [ ]:
# =============================================================================
# 1. SETUP AND DOWNLOAD
# =============================================================================

# For Google Colab - set up Kaggle credentials
try:
    from google.colab import userdata
    import os
    os.environ['KAGGLE_USERNAME'] = userdata.get('KAGGLE_USERNAME')
    os.environ['KAGGLE_KEY'] = userdata.get('KAGGLE_KEY')
    print("Running on Google Colab - Kaggle credentials loaded")
    colab_environment = True
except:
    print("Running on Local env")
    colab_environment = False

# Install kaggle
if colab_environment:
    !pip install kaggle -q

# Download the dataset
print("Download the dataset...")
dataset_name = "andrewmvd/sp-500-stocks"

if colab_environment:
    !kaggle datasets download -d {dataset_name} -p /content/ --unzip -q
    data_path = "/content/"
else:
    # For Jupyter notebook
    !kaggle datasets download -d {dataset_name} --unzip -q
    data_path = "./"

print("Dataset downloaded successfully!")

Running on Google Colab - Kaggle credentials loaded

DOWNLOADING DATASET...
Dataset URL: https://www.kaggle.com/datasets/andrewmvd/sp-500-stocks
License(s): CC0-1.0
Dataset downloaded successfully!


### File Exploration

In [ ]:
#################################################
# FILE EXPLORATION

print(f"\FILES IN DATASET:")

import os
csv_files = []
for file in os.listdir(data_path):
    if file.endswith('.csv'):
        file_path = os.path.join(data_path, file)
        file_size = os.path.getsize(file_path) / 1024 / 1024  # MB
        csv_files.append(file)
        print(f"  {file}")
        print(f"   Size: {file_size:.2f} MB")

        # Get 3 rows
        temp_df = pd.read_csv(file_path, nrows=3)
        print(f"   Columns: {list(temp_df.columns)}")
        print(f"   Sample shape: {temp_df.shape}")
        print()



📁 FILES IN DATASET:
------------------------------
📄 sp500_stocks.csv
   Size: 91.85 MB
   Columns: ['Date', 'Symbol', 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
   Sample shape: (3, 8)

📄 sp500_index.csv
   Size: 0.05 MB
   Columns: ['Date', 'S&P500']
   Sample shape: (3, 2)

📄 sp500_companies.csv
   Size: 0.77 MB
   Columns: ['Exchange', 'Symbol', 'Shortname', 'Longname', 'Sector', 'Industry', 'Currentprice', 'Marketcap', 'Ebitda', 'Revenuegrowth', 'City', 'State', 'Country', 'Fulltimeemployees', 'Longbusinesssummary', 'Weight']
   Sample shape: (3, 16)



In [ ]:
######################
# 3. LOAD MAIN DATASET
######################

print("LOADING MAIN DATASET:")
print("-" * 30)

# Load the main stocks file
main_file = "sp500_stocks.csv"
if main_file in csv_files:
    df_sp500_kaggle = pd.read_csv(os.path.join(data_path, main_file))
    print(f"Loaded {main_file}")
else:
    # If different name, load the first CSV file
    df_sp500_kaggle = pd.read_csv(os.path.join(data_path, csv_files[0]))
    main_file = csv_files[0]
    print(f"Loaded {csv_files[0]} (main file)")

### Load MAIN File (re)

In [ ]:

# load dataset

print(f"\nDATASET OVERVIEW:")
print("-" * 30)
print(f"Shape: {df_sp500_kaggle.shape[0]:,} rows × {df_sp500_kaggle.shape[1]} columns")
print(f"Memory Usage: {df_sp500_kaggle.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")
print(f"Date Range: {df_sp500_kaggle['Date'].min()} to {df_sp500_kaggle['Date'].max()}")

# Convert Date column
df_sp500_kaggle['Date'] = pd.to_datetime(df_sp500_kaggle['Date'])
date_range_days = (df_sp500_kaggle['Date'].max() - df_sp500_kaggle['Date'].min()).days
print(f"Total Days Covered: {date_range_days:,} days")
print(f"Unique Trading Days: {df_sp500_kaggle['Date'].nunique():,}")

In [ ]:
#########################################
# Use S&P symbols to query Yahoo finance

import yfinance as yf
import pandas as pd
from datetime import datetime

# load symbols from sp500_companies.csv
companies = pd.read_csv('sp500_companies.csv')
symbols = companies['Symbol'].tolist()

# define date range
start_date = '2010-01-01'
end_date = datetime.now().strftime('%Y-%m-%d')

# Fetch data
data = yf.download(symbols, start=start_date, end=end_date, group_by='ticker', progress=True)

In [ ]:
##########################
# Kaggle data reshape

Reshape to long format (like Kaggle: Date, Symbol, Open, High, Low, Close, Adj Close, Volume)
df_list = []
for symbol in symbols:
    temp = data[symbol].copy()
    temp['Symbol'] = symbol
    temp = temp.reset_index()[['Date', 'Symbol', 'Open', 'High', 'Low', 'Close', 'Volume']]
    df_list.append(temp)
df = pd.concat(df_list, ignore_index=True)
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
#########################
# column analysis

print(f"\COLUMN DETAILS:")

# Column info table
column_info = pd.DataFrame({
    'Column': df.columns,
    'Data Type': df.dtypes,
    'Non-Null Count': df.count(),
    'Null Count': df.isnull().sum(),
    'Null %': (df.isnull().sum() / len(df) * 100).round(2),
    'Unique Values': [df[col].nunique() for col in df.columns],
    'Memory Usage (KB)': (df.memory_usage(deep=True, index=False) / 1024).round(2),
    'Memory Usage (GB)': (df.memory_usage(deep=True, index=False) / (1024 ** 3)).round(4)
})

print(column_info.to_string(index=False))


## Handle Missing Values

In [ ]:
# Report missing values
print("Missing values:")
print(df.isnull().sum())

price_cols = ['Open', 'High', 'Low', 'Close']

# Calculate % missing volume per stock
missing_pct_volume = df.groupby('Symbol')['Volume'].apply(lambda x: x.isna().mean())

# Get list of stocks with > 10% missing volume
stocks_with_missing_volume_10pct = missing_pct_volume[missing_pct_volume > 0.1].index.tolist()
print(f"\nStocks Missing volume > 10%: {stocks_with_missing_volume_10pct}")
print(f"\nNumber of stocks Missing volume > 10%: {len(stocks_with_missing_volume_10pct)}")

# Get list of stocks with > 05% missing volume
stocks_with_missing_volume_05pct = missing_pct_volume[missing_pct_volume > 0.05].index.tolist()
print(f"\nStocks Missing volume > 05%: {stocks_with_missing_volume_05pct}")
print(f"\nNumber of stocks Missing volume > 05%: {len(stocks_with_missing_volume_05pct)}")


# Drop stocks with excessive missing data
# Columns to review for missing data
price_cols = ['Open', 'High', 'Low', 'Close']
# find missing data in price columns and average across each stock
missing_pct_stock_px = df.groupby('Symbol').apply(lambda x: x[price_cols].isnull().mean()).mean(axis=1)
# display(missing_pct_stock_px)

# find list of stocks missing more than 10%
stocks_to_drop = missing_pct_stock_px[missing_pct_stock_px > 0.1].index
print(f"\nstocks_to_drop: {stocks_to_drop}")
print(f"\nNumber of stocks to drop: {len(stocks_to_drop)}")

# # 05%
stocks_to_drop_05pct = missing_pct_stock_px[missing_pct_stock_px > 0.05].index
print(f"\nstocks_to_drop_05pct: {stocks_to_drop_05pct}")
print(f"\nNumber of stocks to drop: {len(stocks_to_drop_05pct)}")

df = df[~df['Symbol'].isin(stocks_to_drop_05pct)]

In [ ]:
# Report missing values
print("Missing values:")
print(df.isnull().sum())

# Find stocks that have any missing Volume
stocks_to_drop_no_vol = df.groupby('Symbol')['Volume'].apply(lambda x: x.isna().any())
stocks_to_drop_no_vol = stocks_to_drop_no_vol[stocks_to_drop_no_vol].index.tolist()

print("Dropping stocks (any missing Volume):", stocks_to_drop_no_vol)

# Drop them from the dataframe
df = df[~df['Symbol'].isin(stocks_to_drop_no_vol)]

# Verify
print("Missing values:")
print(df.isnull().sum())

## Save Cleaned Stocks to File

In [ ]:
# Save cleaned dataset
df.to_csv('cleaned_sp500_stocks.csv', index=False)
print("Cleaned dataset saved.")

# Feature engineering:

In [ ]:
# Mount your Google Drive again
from google.colab import drive
drive.mount('/content/drive')

# Imports
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/cleaned_sp500_stocks.csv')

# review
print("File loaded successfully!")
print(df.head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
File loaded successfully!
         Date Symbol      Open      High       Low     Close       Volume  \
0  2010-01-04   AAPL  6.407195  6.439316  6.375674  6.424606  493729600.0   
1  2010-01-05   AAPL  6.442317  6.472037  6.401789  6.435712  601904800.0   
2  2010-01-06   AAPL  6.435712  6.461229  6.326739  6.333344  552160000.0   
3  2010-01-07   AAPL  6.356760  6.364265  6.275706  6.321636  477131200.0   
4  2010-01-08   AAPL  6.313230  6.364264  6.276006  6.363664  447610800.0   

   Year  
0  2010  
1  2010  
2  2010  
3  2010  
4  2010  


In [ ]:
# Report missing values
print("Missing values:")
print(df.isnull().sum())

Missing values after first drops:
Date      0
Symbol    0
Open      0
High      0
Low       0
Close     0
Volume    0
Year      0
dtype: int64


### Return

Open-to-Close intraday return:
* Measures how much the stock moved within the day (from opening price to closing price).
* Captures intraday behavior — how bullish or bearish the trading day was.

In [ ]:
##################################
# 'Return' with (Close - Open) / Open
df['Return'] = (df['Close'] - df['Open']) / df['Open']

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1679818 entries, 0 to 1679817
Data columns (total 9 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   Date    1679818 non-null  object 
 1   Symbol  1679818 non-null  object 
 2   Open    1679818 non-null  float64
 3   High    1679818 non-null  float64
 4   Low     1679818 non-null  float64
 5   Close   1679818 non-null  float64
 6   Volume  1679818 non-null  float64
 7   Year    1679818 non-null  int64  
 8   Return  1679818 non-null  float64
dtypes: float64(6), int64(1), object(2)
memory usage: 115.3+ MB


### Annualized Volatility

In [ ]:
# Sort by Symbol and Date
df = df.sort_values(['Symbol', 'Date']).reset_index(drop=True)

# Compute annualized vol per stock
Annual_Vol = df.groupby('Symbol')['Return'].transform(lambda x: x.std() * np.sqrt(252))

# Add new column
df['Annual_Vol'] = Annual_Vol

print(df[['Date', 'Symbol', 'Return', 'Annual_Vol']].head(20))

print("Annualized Volatility per Symbol:")
print(Annual_Vol.head())

          Date Symbol    Return  Annual_Vol
0   2010-01-04      A -0.002867     0.23301
1   2010-01-05      A -0.008010     0.23301
2   2010-01-06      A  0.000000     0.23301
3   2010-01-07      A  0.000975     0.23301
4   2010-01-08      A  0.005222     0.23301
5   2010-01-11      A -0.001943     0.23301
6   2010-01-12      A -0.003599     0.23301
7   2010-01-13      A  0.007220     0.23301
8   2010-01-14      A  0.018307     0.23301
9   2010-01-15      A -0.025304     0.23301
10  2010-01-19      A  0.014493     0.23301
11  2010-01-20      A  0.003275     0.23301
12  2010-01-21      A -0.015484     0.23301
13  2010-01-22      A -0.038880     0.23301
14  2010-01-25      A  0.001020     0.23301
15  2010-01-26      A -0.002724     0.23301
16  2010-01-27      A -0.001027     0.23301
17  2010-01-28      A -0.015105     0.23301
18  2010-01-29      A -0.031110     0.23301
19  2010-02-01      A  0.034446     0.23301
Annualized Volatility per Symbol:
0    0.23301
1    0.23301
2    0.23301
3  

In [ ]:
# Report missing values
print("Missing values after adding Annualized Volatility:")
print(df.isnull().sum())

Missing values after adding Annualized Volatility:
Date          0
Symbol        0
Open          0
High          0
Low           0
Close         0
Volume        0
Year          0
Return        0
Annual_Vol    0
dtype: int64


### Rolling 30 Day Volatility

In [ ]:
df['Roll_Vol_30d'] = (
    df.groupby('Symbol')['Return']
      .transform(lambda x: x.rolling(window=30, min_periods=5).std() * np.sqrt(252))
)

print(df[['Date', 'Symbol', 'Return', 'Roll_Vol_30d']].head(40))


          Date Symbol    Return  Roll_Vol_30d
0   2010-01-04      A -0.002867           NaN
1   2010-01-05      A -0.008010           NaN
2   2010-01-06      A  0.000000           NaN
3   2010-01-07      A  0.000975           NaN
4   2010-01-08      A  0.005222      0.077859
5   2010-01-11      A -0.001943      0.069944
6   2010-01-12      A -0.003599      0.065582
7   2010-01-13      A  0.007220      0.077847
8   2010-01-14      A  0.018307      0.122781
9   2010-01-15      A -0.025304      0.178262
10  2010-01-19      A  0.014493      0.184658
11  2010-01-20      A  0.003275      0.176554
12  2010-01-21      A -0.015484      0.183352
13  2010-01-22      A -0.038880      0.239618
14  2010-01-25      A  0.001020      0.231588
15  2010-01-26      A -0.002724      0.223739
16  2010-01-27      A -0.001027      0.216770
17  2010-01-28      A -0.015105      0.215198
18  2010-01-29      A -0.031110      0.231925
19  2010-02-01      A  0.034446      0.265692
20  2010-02-02      A  0.009583   

In [ ]:
# Report missing values
print("Missing values after adding 30 Day rolling Volatility:")
print(df.isnull().sum())

Missing values after adding 30 Day rolling Volatility:
Date               0
Symbol             0
Open               0
High               0
Low                0
Close              0
Volume             0
Year               0
Return             0
Annual_Vol         0
Roll_Vol_30d    1708
dtype: int64


### Returns: Lagged, Close-to-close

In [ ]:
# Close to close return or daily return
df['Return_Daily'] = df['Close'].pct_change()


In [ ]:
# Lagged Returns (Return_t-1, Return_t-2)
df['Return_t-1'] = df.groupby('Symbol')['Return'].shift(1)
df['Return_t-2'] = df.groupby('Symbol')['Return'].shift(2)

In [ ]:
# Report missing values
print("Missing values after adding Return lagged, close2close:")
print(df.isnull().sum())

Missing values after adding Return lagged, close2close:
Date               0
Symbol             0
Open               0
High               0
Low                0
Close              0
Volume             0
Year               0
Return             0
Annual_Vol         0
Roll_Vol_30d    1708
Return_Daily       1
Return_t-1       427
Return_t-2       854
dtype: int64


In [ ]:
# Save cleaned dataset
df.to_csv('/content/drive/MyDrive/sp500_stocks_feature-eng.csv', index=False)

print("Complete dataset saved to your Google Drive.")


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Import
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/sp500_stocks_feature-eng.csv')

# reviw
print("sp500_stocks_feature-eng.csv loaded successfully!")
print(df.head())


Mounted at /content/drive
sp500_stocks_feature-eng.csv loaded successfully!
         Date Symbol       Open       High        Low      Close     Volume  \
0  2010-01-04      A  19.988936  20.141767  19.823370  19.931625  3815561.0   
1  2010-01-05      A  19.874313  19.880681  19.587755  19.715115  4186031.0   
2  2010-01-06      A  19.645065  19.740584  19.587753  19.645065  3243779.0   
3  2010-01-07      A  19.600485  19.625958  19.422184  19.619591  3095172.0   
4  2010-01-08      A  19.511342  19.645069  19.358512  19.613228  3733918.0   

   Year    Return  Annual_Vol  Roll_Vol_30d  Return_Daily  Return_t-1  \
0  2010 -0.002867     0.23301           NaN           NaN         NaN   
1  2010 -0.008010     0.23301           NaN     -0.010863   -0.002867   
2  2010  0.000000     0.23301           NaN     -0.003553   -0.008010   
3  2010  0.000975     0.23301           NaN     -0.001297    0.000000   
4  2010  0.005222     0.23301      0.077859     -0.000324    0.000975   

   Return_

### Moving Averages

In [ ]:
# Sort by Symbol and Date
df = df.sort_values(['Symbol', 'Date']).reset_index(drop=True)

In [ ]:
print("Adding simple moving averages...")

# Moving Averages (5, 10, 20, 50, 200)
df['MA_5'] = df.groupby('Symbol')['Close'].transform(
    lambda x: x.rolling(window=5, min_periods=1).mean()
)

df['MA_10'] = df.groupby('Symbol')['Close'].transform(
    lambda x: x.rolling(window=10, min_periods=1).mean()
)

df['MA_20'] = df.groupby('Symbol')['Close'].transform(
    lambda x: x.rolling(window=20, min_periods=1).mean()
)

df['MA_50'] = df.groupby('Symbol')['Close'].transform(
    lambda x: x.rolling(window=50, min_periods=1).mean()
)

df['MA_200'] = df.groupby('Symbol')['Close'].transform(
    lambda x: x.rolling(window=200, min_periods=1).mean()
)

# Price relative to moving averages
df['Price_to_MA20'] = df['Close'] / df['MA_20']
df['Price_to_MA50'] = df['Close'] / df['MA_50']

print("Simple Moving averages completed!")

Adding simple moving averages...
Simple Moving averages completed!


In [ ]:
# Report missing values
print("Missing values after adding Simple Moving Averages (SMA):")
print(df.isnull().sum())

Missing values after adding Simple Moving Averages (SMA):
Date                0
Symbol              0
Open                0
High                0
Low                 0
Close               0
Volume              0
Year                0
Return              0
Annual_Vol          0
Roll_Vol_30d     1708
Return_Daily        1
Return_t-1        427
Return_t-2        854
MA_5                0
MA_10               0
MA_20               0
MA_50               0
MA_200              0
Price_to_MA20       0
Price_to_MA50       0
dtype: int64


In [ ]:
# Exponential Moving Averages (EMA) — reacts faster to price changes
df['EMA_12'] = df.groupby('Symbol')['Close'].transform(lambda x: x.ewm(span=12, adjust=False).mean())
df['EMA_26'] = df.groupby('Symbol')['Close'].transform(lambda x: x.ewm(span=26, adjust=False).mean())


In [ ]:
print("Adding Exponential Moving Averages...")

# Exponential Moving Averages (different timeframes)

df['EMA_5'] = df.groupby('Symbol')['Close'].transform(
    lambda x: x.ewm(span=5).mean()
)

df['EMA_10'] = df.groupby('Symbol')['Close'].transform(
    lambda x: x.ewm(span=10).mean()
)

#12-day EMA & 26-day EMA:
df['EMA_12'] = df.groupby('Symbol')['Close'].transform(
    lambda x: x.ewm(span=12).mean()
)

df['EMA_20'] = df.groupby('Symbol')['Close'].transform(
    lambda x: x.ewm(span=20).mean()
)

#12-day EMA & 26-day EMA:
df['EMA_26'] = df.groupby('Symbol')['Close'].transform(
    lambda x: x.ewm(span=26).mean()
)

# 50-day EMA:
df['EMA_50'] = df.groupby('Symbol')['Close'].transform(
    lambda x: x.ewm(span=50).mean()
)

# Price relative to EMAs
df['Price_to_EMA10'] = df['Close'] / df['EMA_10']
df['Price_to_EMA20'] = df['Close'] / df['EMA_20']
df['Price_to_EMA50'] = df['Close'] / df['EMA_50']

# EMA crossover signals
df['EMA_Cross_5_10'] = df['EMA_5'] - df['EMA_10']
df['EMA_Cross_10_20'] = df['EMA_10'] - df['EMA_20']

print("Exponential Moving Averages, price rations and crossover signals completed!")

Adding Exponential Moving Averages...
Exponential Moving Averages, price rations and crossover signals completed!


In [ ]:
# Report missing values
print("Missing values after adding Exp Moving Averages (EMA):")
print(df.isnull().sum())

Missing values after adding Exp Moving Averages (EMA):
Date                  0
Symbol                0
Open                  0
High                  0
Low                   0
Close                 0
Volume                0
Year                  0
Return                0
Annual_Vol            0
Roll_Vol_30d       1708
Return_Daily          1
Return_t-1          427
Return_t-2          854
MA_5                  0
MA_10                 0
MA_20                 0
MA_50                 0
MA_200                0
Price_to_MA20         0
Price_to_MA50         0
EMA_5                 0
EMA_10                0
EMA_12                0
EMA_20                0
EMA_26                0
EMA_50                0
Price_to_EMA10        0
Price_to_EMA20        0
Price_to_EMA50        0
EMA_Cross_5_10        0
EMA_Cross_10_20       0
dtype: int64


### Momentum

In [ ]:
##################
# MACD (Moving Average Convergence Divergence)

print("Calculating MACD (using existing EMAs)...")

# MACD (Moving Average Convergence Divergence)
df['MACD'] = df['EMA_12'] - df['EMA_26']

df['MACD_Signal'] = df.groupby('Symbol')['MACD'].transform(
    lambda x: x.ewm(span=9).mean()
)

df['MACD_Histogram'] = df['MACD'] - df['MACD_Signal']

print("MACD indicators completed!")

Calculating MACD (using existing EMAs)...
MACD indicators completed!


In [ ]:
###########################
# RSI

print("Calculating RSI...")

def calculate_rsi(group, window=14):
    """Calculate RSI for a price series"""
    delta = group.diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

df['RSI'] = df.groupby('Symbol')['Close'].transform(calculate_rsi)

print("RSI calculation completed!")

Calculating RSI...
RSI calculation completed!


In [ ]:
# Report missing values
print("Missing values after adding Momentum indicators")
print(df.isnull().sum())

Missing values after adding Momentum indicators
Date                  0
Symbol                0
Open                  0
High                  0
Low                   0
Close                 0
Volume                0
Year                  0
Return                0
Annual_Vol            0
Roll_Vol_30d       1708
Return_Daily          1
Return_t-1          427
Return_t-2          854
MA_5                  0
MA_10                 0
MA_20                 0
MA_50                 0
MA_200                0
Price_to_MA20         0
Price_to_MA50         0
EMA_5                 0
EMA_10                0
EMA_12                0
EMA_20                0
EMA_26                0
EMA_50                0
Price_to_EMA10        0
Price_to_EMA20        0
Price_to_EMA50        0
EMA_Cross_5_10        0
EMA_Cross_10_20       0
MACD                  0
MACD_Signal           0
MACD_Histogram        0
RSI                 688
dtype: int64


### Volume features

In [ ]:
import numpy as np

print("Adding volume features...")

# Volume moving averages
df['Volume_MA_10'] = df.groupby('Symbol')['Volume'].transform(
    lambda x: x.rolling(window=10, min_periods=1).mean()
)

df['Volume_MA_30'] = df.groupby('Symbol')['Volume'].transform(
    lambda x: x.rolling(window=30, min_periods=5).mean()
)

# Volume ratios (current volume vs average)
df['Volume_Ratio_10d'] = df['Volume'] / df['Volume_MA_10']

# Price-Volume relationship
df['PV_Trend'] = df['Return'] * np.log(df['Volume'] + 1)  # Adding 1 to avoid log(0)

print("Volume features completed!")

Adding volume features...
Volume features completed!


### Risk measures

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

# Define constants
ANNUAL_TRADING_DAYS = 252
RISK_FREE_RATE_ANNUAL = 0.02  # 2% annualized
RISK_FREE_RATE_DAILY = RISK_FREE_RATE_ANNUAL / ANNUAL_TRADING_DAYS  # ~0.0000794

# Calculate rolling 252-day Sharpe Ratio
def calculate_sharpe(group):
    # rolling mean return (252d)
    mean_return = group['Return'].rolling(window=ANNUAL_TRADING_DAYS, min_periods=20).mean()
    # rolling standard deviation of return (252d)
    std_return = group['Return'].rolling(window=ANNUAL_TRADING_DAYS, min_periods=20).std()
    # Sharpe Ratio: (Mean Return - Risk-Free Rate) / Std Return
    sharpe = (mean_return - RISK_FREE_RATE_DAILY) / std_return
    return sharpe

# Apply to each stock
df['Sharpe_252d'] = df.groupby('Symbol').apply(calculate_sharpe, include_groups=False).reset_index(drop=True)

# Verify no missing values
print("Missing values after adding Sharpe_252d:")
print(df.isna().sum())

# Summary of Sharpe_252d
print("\nSharpe_252d summary:")
print(df['Sharpe_252d'].describe())

Missing values after adding Sharpe_252d:
Date                   0
Symbol                 0
Open                   0
High                   0
Low                    0
Close                  0
Volume                 0
Year                   0
Return                 0
Annual_Vol             0
Roll_Vol_30d        1708
Return_Daily           1
Return_t-1           427
Return_t-2           854
MA_5                   0
MA_10                  0
MA_20                  0
MA_50                  0
MA_200                 0
Price_to_MA20          0
Price_to_MA50          0
EMA_5                  0
EMA_10                 0
EMA_12                 0
EMA_20                 0
EMA_26                 0
EMA_50                 0
Price_to_EMA10         0
Price_to_EMA20         0
Price_to_EMA50         0
EMA_Cross_5_10         0
EMA_Cross_10_20        0
MACD                   0
MACD_Signal            0
MACD_Histogram         0
RSI                  688
Volume_MA_10           0
Volume_MA_30        1708
Volume_Ra

### Join Sector/Company data:

In [ ]:
import pandas as pd

# Load from uploaded file
companies = pd.read_csv('/content/sp500_companies.csv')


# Verify loading
print(companies.head())
print(companies.columns)
print(f"Unique symbols: {companies['Symbol'].nunique()}")

  Exchange Symbol              Shortname               Longname  \
0      NMS   AAPL             Apple Inc.             Apple Inc.   
1      NMS   NVDA     NVIDIA Corporation     NVIDIA Corporation   
2      NMS   MSFT  Microsoft Corporation  Microsoft Corporation   
3      NMS   AMZN       Amazon.com, Inc.       Amazon.com, Inc.   
4      NMS  GOOGL          Alphabet Inc.          Alphabet Inc.   

                   Sector                        Industry  Currentprice  \
0              Technology            Consumer Electronics        254.49   
1              Technology                  Semiconductors        134.70   
2              Technology       Software - Infrastructure        436.60   
3       Consumer Cyclical                 Internet Retail        224.92   
4  Communication Services  Internet Content & Information        191.41   

       Marketcap        Ebitda  Revenuegrowth           City State  \
0  3846819807232  1.346610e+11          0.061      Cupertino    CA   
1  329

In [ ]:
#join sector/company data
df = df.merge(companies[['Symbol', 'Sector', 'Industry', 'Longname']], on='Symbol', how='left')

# Verify no missing values
print("Missing values after adding Sharpe_252d:")
print(df.isna().sum())

Missing values after adding Sharpe_252d:
Date                   0
Symbol                 0
Open                   0
High                   0
Low                    0
Close                  0
Volume                 0
Year                   0
Return                 0
Annual_Vol             0
Roll_Vol_30d        1708
Return_Daily           1
Return_t-1           427
Return_t-2           854
MA_5                   0
MA_10                  0
MA_20                  0
MA_50                  0
MA_200                 0
Price_to_MA20          0
Price_to_MA50          0
EMA_5                  0
EMA_10                 0
EMA_12                 0
EMA_20                 0
EMA_26                 0
EMA_50                 0
Price_to_EMA10         0
Price_to_EMA20         0
Price_to_EMA50         0
EMA_Cross_5_10         0
EMA_Cross_10_20        0
MACD                   0
MACD_Signal            0
MACD_Histogram         0
RSI                  688
Volume_MA_10           0
Volume_MA_30        1708
Volume_Ra

### Target/Y - column

In [ ]:
##################################
# create target

# sort by Symbol and Date
df = df.sort_values(by=["Symbol", "Date"])

# Target: Next-Day Return (Regression)
# percentage chg from the close of the current day (T) to the close of the next day (T+1)
df["Target_Return_Next_Day"] = df.groupby("Symbol")["Close"].pct_change().shift(-1)

# Target: Price Direction (Classification)
# 1 if the next day's return is positive, 0 otherwise.
df["Target_Direction_Next_Day"] = (df["Target_Return_Next_Day"] > 0).astype(int)

# Target: Next-Week Return (Multi-period Regression)
# percentage chg from the close of the current day (T) to the close 5 days in the future (T+5)
df["Target_Return_Next_Week"] = df.groupby("Symbol")["Close"].pct_change(periods=5).shift(-5)

# Clean up rows where targets cannot be calculated
# This will be the last 5 rows for each symbol, as they lack future data for the weekly target.
df = df.dropna(subset=["Target_Return_Next_Day", "Target_Return_Next_Week"])

# --- Display the results ---
print("Generated Targets:")
print(df[["Symbol", "Date", "Close", "Target_Return_Next_Day", "Target_Direction_Next_Day", "Target_Return_Next_Week"]].head(10))

print("\nLast few rows for a sample symbol to show NaN drop:")
print(df[df["Symbol"] == "AAPL"][["Symbol", "Date", "Close", "Target_Return_Next_Day", "Target_Return_Next_Week"]].tail(10))

Generated Targets:
  Symbol       Date      Close  Target_Return_Next_Day  \
0      A 2010-01-04  19.931625               -0.010863   
1      A 2010-01-05  19.715115               -0.003553   
2      A 2010-01-06  19.645065               -0.001297   
3      A 2010-01-07  19.619591               -0.000324   
4      A 2010-01-08  19.613228                0.000649   
5      A 2010-01-11  19.625954               -0.012005   
6      A 2010-01-12  19.390343                0.007882   
7      A 2010-01-13  19.543177                0.014989   
8      A 2010-01-14  19.836102               -0.023114   
9      A 2010-01-15  19.377609                0.012159   

   Target_Direction_Next_Day  Target_Return_Next_Week  
0                          0                -0.015336  
1                          0                -0.016473  
2                          0                -0.005186  
3                          0                 0.011035  
4                          1                -0.012013  
5     

# Save Final file

In [ ]:
# Save dataset
df.to_csv('/content/drive/MyDrive/sp500_stocks_feature-eng.csv', index=False)
